In [1]:
import pandas as pd
import re
from re import compile, search
import warnings
warnings.filterwarnings('ignore')

In [2]:
file_name = 'sample_100_pages.json'
df_sample = pd.read_json(file_name)

In [3]:
regexps = {
    'name': compile('име обект: (?P<name>.*?) вид обект'),
    'category': compile('вид обект: (?P<category>.*?) град'),
    'city': compile('град: (?P<city>.*?) адрес'),
    'address': compile('адрес: (?P<address>.*?) описание')
}

def match(prop, text):
    m = regexps[prop].search(text)
    if m is not None:
        return m.group(prop).strip()
    else:
        return None

def match_name(text):
    return match('name', text)

In [4]:
df_sample['matched_name'] = df_sample['description'].apply(lambda x: match('name', x))
df_sample['matched_category'] = df_sample['description'].apply(lambda x: match('category', x))
df_sample['matched_city'] = df_sample['description'].apply(lambda x: match('city', x))
df_sample['matched_address'] = df_sample['description'].apply(lambda x: match('address', x))

In [5]:
df_sample['matched_name'].count()

883

In [6]:
df_sample['matched_city'].count()

885

In [7]:
df_sample['matched_address'].count()

879

In [8]:
df_sample['matched_category'].count()

883

In [9]:
categories = list(filter(lambda x: x is not None, df_sample['matched_category'].unique()))

cat_group = "|".join(categories)
regex = "(?P<name>({}).*?),".format(cat_group)

r = compile(regex, flags=re.IGNORECASE)

def match_title(title):
    m = r.search(title)
    if m is not None:
        return m.group('name')

for row in df_sample['title'].apply(match_title).unique():
    print(row)
    
    


сладкарница Малинка
None
ресторант  "Mr. Pizza"
рсторант "Галерия"
заведение SARAY turkish restaurant
ресторант "Casita food & wine"
кафе бар "Табакера"
ресторант "Етно"
ресторант "На улицата"
клуб "Noir"
заведение Златна Белка
клуб Десепрадо
пиано бар Poison
БАР
бар Самис
автобус 27
пиано бар Силикон
заведение
Bar съзнателно нарушава забраната за пушене
бистро Венера
игрална зала "Мистрал"
кафе
кафе Cache
бистро Венеция
пицария Ралица
механа Кукуш
заведение за бързо хранене Мимас
ресторант Ниагара
бар Кариби
ресторант "Art club museum "
Бар енд динър "Бар Глори"
кафене "Зодиак"
Пицария Ветрило
кафене
вход на жилищна сграда
Бар
механа "Роден край"
Ресторант "Sweet"
Bar & Grill Младост цигарен дим
Ресторант "Спортела"
заведение "Магурата"
Пиано бар "Гетсби"
Club
Ресторант "Grillo"
заведение "Мода бар"
ресторанти  "Sweet bar and grill"
Ресторант  "Московска 15"
ресторант "The view"
ресторант "Къщата"
ресторанта на Арена Асикс
нощен клуб "БИАД"
пиано бар "Cheers"
ресторант "Sweet bar and 

In [10]:
# df_sample['matched_title'] = df_sample['title'].apply(match_title)

In [11]:
# df_sample[['id', 'matched_title', 'matched_name']].dropna(how='all')

In [12]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import re

def parse_category(category):
    splitted = re.split(r'[-\\\/,]', category)
    return [w.strip().lower() for w in splitted]

def flatten(l):
    return sum(l, [])

all_cats = flatten([ parse_category(category) for category in categories])
unique_cats = set(all_cats)
unique_cats

{'bar',
 'bar and dinner',
 'bar and food',
 'bar and grill',
 'bar&dinner',
 'bar&grill',
 'cafeteria & gelateria & pasticeria',
 'club',
 'diner & bar',
 'kafe bar',
 'kafene',
 'pizza & restaurant',
 'playground',
 'автобус',
 'автобусна линия 83',
 'административна сграда',
 'аператив',
 'аператив  център',
 'аперитив',
 'аперитив и магазин',
 'арабски ресторант',
 'бaр',
 'бар',
 'бар билярд',
 'бар енд динър',
 'бар и грил',
 'бар и кафе',
 'бар и клуб',
 'бар и ресторант',
 'бар и хапване',
 'бар кафе',
 'бар клуб',
 'бар ресторант',
 'барбекю',
 'барове',
 'бензиностанция',
 'билярд клуб',
 'бирария',
 'бирария и механа',
 'бистро',
 'бистро (кръчма)',
 'бистро и бензиностанция',
 'бистро+бензиностанция',
 'болница',
 'боулинг',
 'боулинг бар',
 'висше учебно зaведение',
 'вход',
 'входа на блока',
 'детска градина',
 'детска площадка',
 'детската площадка',
 'дискотекa',
 'дискотека',
 'една от големите зали на приземен етаж',
 'електронно казино',
 'жилищен блок',
 'заведение

In [13]:
import nltk
from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk.tag import pos_tag, RegexpTagger
from nltk.chunk.regexp import RegexpChunkParser, RegexpChunkRule
from nltk.tree import Tree

cat_group = "|".join(unique_cats)
regexp_tagger = RegexpTagger([
    ("^{}$".format(cat_group), 'PLACE_MARKER'),
    (r'^[,]$', 'DT'),
    (r'^в$', 'BEG'),
    (r'^[A-Za-z0-9 &]+$', 'LATIN_WORD'),
    (r'^[а-яА-Я0-9 &]+$', 'CYRILLIC_WORD')
])

tweet_token = TweetTokenizer()

def tokenize(title):
    words = tweet_token.tokenize(title)
    tokens = regexp_tagger.tag(words)
    return [token for token in tokens if token[1] is not None]

def tokenize2(title):
    words = word_tokenize(title)
    tokens = regexp_tagger.tag(words)
    return [token for token in tokens if token[1] is not None]

chuncker = RegexpChunkParser([
        RegexpChunkRule.fromstring(r'{<BEG> (<LATIN_WORD>|<CYRILLIC_WORD>)+ <PLACE_MARKER>+ (<LATIN_WORD>|<CYRILLIC_WORD>)* (?=<DT>|<BEG>)*}'),
        RegexpChunkRule.fromstring(r'{(<PLACE_MARKER>+ (<LATIN_WORD>|<CYRILLIC_WORD>)+)+ <PLACE_MARKER>* (?=<DT>|<BEG>)*}'),
        RegexpChunkRule.fromstring(r'{<PLACE_MARKER>+ (<LATIN_WORD>|<CYRILLIC_WORD>)+ (?=<DT>|<BEG>)*}'),
    ],
    chunk_label='Name'
)


def get_words(tree):
    return [word[0] for word in tree.leaves()]

def find_name(title):
    if title is "":
        return None
    
    title = title.lower()
    tokens = tokenize(title)
    tree = chuncker.parse(tokens)
    chunk_tree = [get_words(t) for t in tree if type(t) is Tree]
    flattened = sum(chunk_tree, [])
    filtered = [w.strip() for w in flattened if w != "в"]
    return " ".join(filtered) if filtered is not [] else None

def find_name2(title):
    if title is "":
        return None
    
    title = title.lower()
    tokens = tokenize2(title)
    tree = chuncker.parse(tokens)
    chunk_tree = [get_words(t) for t in tree if type(t) is Tree]
    flattened = sum(chunk_tree, [])
    filtered = [w.strip() for w in flattened if w != "в"]
    return " ".join(filtered) if filtered is not [] else None

In [14]:
# 10086 - no comma


In [15]:
df_sample['matched_title'] = df_sample['title'].apply(find_name)

In [16]:
df_sample['matched_title_2'] = df_sample['title'].apply(find_name2)

In [17]:
# df_sample['matched_title_2'] = df_sample['title'].apply(lambda r: find_name(r, chuncker2))

In [18]:
df_sample[df_sample.matched_title != df_sample.matched_title_2][['id', 'matched_title', 'matched_title_2']]

,id,matched_title,matched_title_2
2,10355,ресторант mr pizza,ресторант pizza
78,9917,None,None
105,10036,ресторант mr pizza,ресторант pizza
112,10048,кафе бар monroe acoustic caffe,кафе/бар monroe acoustic caffe
291,10173,кафе-аперитив магазин рико 33 еоод,кафе-аперитив/магазин еоод
391,10421,кафе аперитив люти чушки,кафе- аперитив люти чушки
435,10372,кафе ресторант хаджиев 11 еоод кафе макси,кафе ресторант хаджиев еоод кафе макси
454,10538,кафе-аперитив и магазин рико 33 еоод,кафе-аперитив и магазин еоод
511,9833,ресторант искър,
644,9447,None,None


In [19]:
# df_sample[['id', 'matched_title', 'matched_title_2']].dropna()

In [20]:
title = 'Пушене в ресторант \'Искър\', град Своге'.lower()
tokens = tokenize2(title)
tree = chuncker.parse(tokens)
chunk_tree = [get_words(t) for t in tree if type(t) is Tree]
flattened = sum(chunk_tree, [])
filtered = [w for w in flattened if w != "в"]

tokens

[('пушене', 'CYRILLIC_WORD'),
 ('в', 'BEG'),
 ('ресторант', 'PLACE_MARKER'),
 (',', 'DT'),
 ('град', 'CYRILLIC_WORD'),
 ('своге', 'CYRILLIC_WORD')]

In [21]:
# df_sample['matched_title_desc'] = df_sample['description'].apply(find_name)

In [22]:
df_sample[['id', 'matched_name', 'matched_title']][100:150]

,id,matched_name,matched_title
100,9931,None,заведение маки
101,9932,None,ресторант sasa paradise mall
102,9933,None,бар cheers
103,9938,None,механа одаята
104,10034,Маки,бирария маки
105,10036,Mr. Pizza,ресторант mr pizza
106,10035,None,ретро бар градини
107,10032,None,
108,10041,None,пицария amaro
109,10039,None,ресторант ариана


In [23]:
df_sample.to_json('sample_100_pages_names.json')

In [24]:
pd.read_json('sample_100_pages_names.json').count()

categories          3000
date                3000
description         3000
files               3000
id                  3000
location            3000
title               3000
matched_name         883
matched_category     883
matched_city         885
matched_address      879
matched_title       2989
matched_title_2     2989
dtype: int64